In [1]:
import numpy as np

In [2]:
input_sentences = []
output_sentences = []

no_of_inputs = 25000
count = 0

for line in open(r'C:\Users\Vamishra\Downloads\mar-eng\mar.txt', encoding="utf-8"):
    
    count += 1
    
    inp = line.rstrip().split('\t')[0]
    out = line.rstrip().split('\t')[1]
    
    input_sentences.append(inp)
    output_sentences.append(out)
    
    if count > no_of_inputs:
        break

    

In [3]:
for _ in range(5):
    j = np.random.choice(no_of_inputs)
    print(input_sentences[j] + "  ---->  " + output_sentences[j])

Don't call me.  ---->  मला कॉल नको करूस.
That's about it.  ---->  तेवढंच.
Was I snoring?  ---->  मी घोरत होतो का?
I'm in Perth.  ---->  मी पर्थमध्ये आहे.
Tom was working for me.  ---->  टॉम माझ्यासाठी काम करत होता.


In [4]:
input_decoder_sentences = []
output_decoder_sentences = []

for i in range(len(output_sentences)):
    inp_dec_sent = 'sos ' + output_sentences[i]
    input_decoder_sentences.append(inp_dec_sent)
    out_dec_sent = output_sentences[i] + ' eos'
    output_decoder_sentences.append(out_dec_sent)
    

In [5]:
print(output_decoder_sentences[1:5])
print(input_decoder_sentences[1:5])

['पळ! eos', 'धाव! eos', 'पळा! eos', 'धावा! eos']
['sos पळ!', 'sos धाव!', 'sos पळा!', 'sos धावा!']


In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input , Embedding , LSTM , Dense
from keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

import warnings

C:\Users\Vamishra\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Vamishra\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Vamishra\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:

In [7]:
encoder_tokens = Tokenizer()
encoder_tokens.fit_on_texts(input_sentences)

In [8]:
eng_word2idx = encoder_tokens.word_index

In [9]:
input_encoder_sentences = encoder_tokens.texts_to_sequences(input_sentences)

In [10]:
decoder_tokens = Tokenizer()
decoder_tokens.fit_on_texts(input_decoder_sentences + output_decoder_sentences)

In [11]:
marathi_word2idx = decoder_tokens.word_index

In [12]:
input_decoder_sentences = decoder_tokens.texts_to_sequences(input_decoder_sentences)
output_decoder_sentences = decoder_tokens.texts_to_sequences(output_decoder_sentences)

In [13]:
max_input_length = max(len(sen) for sen in input_encoder_sentences)
max_output_length = max(len(sen) for sen in output_decoder_sentences)
print(max_input_length)
print(max_output_length)

8
11


In [14]:
input_encoder_sentences = pad_sequences(input_encoder_sentences , maxlen=max_input_length , padding='pre')
input_decoder_sentences = pad_sequences(input_decoder_sentences , maxlen=max_output_length , padding='post')
output_decoder_sentences = pad_sequences(output_decoder_sentences , maxlen=max_output_length , padding='post')

In [15]:
english_vocab_size = len(eng_word2idx) + 1
marathi_vocab_size = len(marathi_word2idx) + 1

In [16]:
one_hot_output_decoder_tokens = np.zeros((len(output_decoder_sentences) , max_output_length , marathi_vocab_size))

In [17]:
for i , j in enumerate(output_decoder_sentences):
    for k , word in enumerate(j):
        one_hot_output_decoder_tokens[i , k , word] = 1
    

In [18]:
print(input_encoder_sentences[103:104])
print(input_decoder_sentences[103:104])
print(one_hot_output_decoder_tokens[103:104][0])

[[  0   0   0   0   0   0 179  11]]
[[  1   6 478   0   0   0   0   0   0   0   0]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [19]:
# building up the model

Embedding_length = 100
lstm_unit = 30

encoding_input = Input(batch_shape=(None , max_input_length))

encoding_embedding = Embedding(english_vocab_size , Embedding_length , input_length=max_input_length)

embedding_out_encode = encoding_embedding(encoding_input)

encoding_lstm = LSTM(lstm_unit , return_state=True)

_ , state_h , state_c = encoding_lstm(embedding_out_encode)


states = [state_h , state_c]

decoding_input = Input(batch_shape=(None , max_output_length))

decoding_embedding = Embedding(marathi_vocab_size , Embedding_length)

embedding_out_decode = decoding_embedding(decoding_input)

decoding_lstm = LSTM(lstm_unit , return_sequences=True , return_state=True)

decoding_output , _ , _ = decoding_lstm(embedding_out_decode , initial_state=states)

decoding_dense = Dense(marathi_vocab_size , activation='softmax')

decoding_output = decoding_dense(decoding_output)

seq2seq_model = Model([encoding_input , decoding_input] , decoding_output)

seq2seq_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 8, 100)       359300      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 11, 100)      775300      input_2[0][0]                    
____________________________________________________________________________________________

In [20]:
seq2seq_model.compile(optimizer='rmsprop' , loss='categorical_crossentropy' , metrics=['accuracy'])

In [21]:
seq2seq_model.fit([input_encoder_sentences , input_decoder_sentences] , one_hot_output_decoder_tokens , epochs=20 , batch_size=200 , validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 20000 samples, validate on 5001 samples
Epoch 1/20
20000/20000 [==============================] - 162s 8ms/step - loss: 6.1812 - accuracy: 0.5579 - val_loss: 4.5930 - val_accuracy: 0.4808
Epoch 2/20
20000/20000 [==============================] - 161s 8ms/step - loss: 3.2941 - accuracy: 0.5633 - val_loss: 3.5229 - val_accuracy: 0.4808
Epoch 3/20
20000/20000 [==============================] - 164s 8ms/step - loss: 2.6307 - accuracy: 0.5662 - val_loss: 3.1937 - val_accuracy: 0.4996
Epoch 4/20
20000/20000 [==============================] - 162s 8ms/step - loss: 2.3527 - accuracy: 0.6427 - val_loss: 3.0079 - val_accuracy: 0.5850
Epoch 5/20
20000/20000 [==============================] - 160s 8ms/step - loss: 2.1988 - accuracy: 0.6678 - val_loss: 2.9433 - val_accuracy: 0.5863
Epoch 6/20
20000/20000 [==============================] - 168s 8ms/step - loss: 2.1093 - accuracy: 0.6733 - val_loss

In [22]:
seq2seq_model.save(r'C:\Users\Vamishra\Downloads\mar-eng\english_to_marathi.h5')

In [23]:
# Creating inference model

encoding_inference = Model(encoding_input , states)

decoder_input_hidden_state = Input(batch_shape=(None , lstm_unit))
decoder_input_cell_state = Input(batch_shape=(None , lstm_unit))

decoder_input_states = [decoder_input_hidden_state , decoder_input_cell_state]

single_input = Input(batch_shape=(1,1))

single_embedding_output = decoding_embedding(single_input)

single_decoder_output , decoder_h_state , decoder_c_state = decoding_lstm(single_embedding_output , initial_state = decoder_input_states)

decoder_states = [decoder_h_state , decoder_c_state]

single_decoder_output = decoding_dense(single_decoder_output)

decoding_inference = Model([single_input] + decoder_input_states , [single_decoder_output] + decoder_states)

encoding_inference.summary()

decoding_inference.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 8, 100)            359300    
_________________________________________________________________
lstm_1 (LSTM)                [(None, 30), (None, 30),  15720     
Total params: 375,020
Trainable params: 375,020
Non-trainable params: 0
_________________________________________________________________
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (1, 1)               0                                            
_____________________________________________________________________

In [24]:
eng_idx2word = {j : i for i , j in eng_word2idx.items()}
marathi_idx2word = {j : i for i,j in marathi_word2idx.items()}

In [25]:
def translate(seq):
    
    states = encoding_inference.predict(seq)
    
    single_inp = np.zeros((1,1))
    
    single_inp[0,0] = marathi_word2idx['sos']
    
    eos_idx = marathi_word2idx['eos']
    
    sentence = []
    
    for _ in range(max_output_length):
        
        output , h , c = decoding_inference.predict([single_inp] + states)
        
        index = np.argmax(output[0,0,:])
        
        if index == eos_idx:
            break
        if index > 0:
            word = marathi_idx2word[index]
            
            sentence.append(word)
        
        states = [h , c]
        
        single_inp[0,0] = index
        
    return ' '.join(sentence)     
        
        
        
        

In [26]:
def beam_search_translate(seq):
    
    states = encoding_inference.predict(seq)
    
    single_inp = np.zeros((1,1))
    
    single_inp[0,0] = marathi_word2idx['sos']
    
    end_idx = marathi_word2idx['eos']
    
    sentence = ''
    
    for _ in range(len(max_output_length)):
        
        output , h , c = decoding_inference.predict([single_inp] + states)
        
        decoder_state = [h ,c]
        
        beam_idx = np.argsort(output[0,0,:])[::-1][0:k]
        
        if beam_idx[0] == end_idx:
            break
            
        else:
            for i in beam_idx:
                if i != end_idx and i > 0:
                    single_inp[0,0] = i
                    output , h , c = decoding_inference.predict([single_inp] + decoder_state)
                    
                    next_beam_idx = np.argsort(output[0,0,:])[::-1][0:k]
                    
                    result_beam = [i*j for j in next_beam_idx]
            
        
        # applying beam search 
        
        

In [36]:
for _ in range(8):
    
    i = np.random.choice(count)
    print("Original English Sentence-->   " + input_sentences[i])
    print("Original Marathi Sentence-->   " + output_sentences[i])
    print("Predicted Marathi Sentence->   " + translate(input_encoder_sentences[i:i+1]))
    print()

Original English Sentence-->   Do you know her?
Original Marathi Sentence-->   तुम्ही त्यांना ओळखता का?
Predicted Marathi Sentence->   तू तुला करू शकत नाही

Original English Sentence-->   Tom calls Mary every night.
Original Marathi Sentence-->   टॉम मेरीला दर रात्री फोन करतो.
Predicted Marathi Sentence->   टॉम टॉमला मेरीला पाहिलं

Original English Sentence-->   Where exactly do you live?
Original Marathi Sentence-->   तू नक्की कुठे राहतेस?
Predicted Marathi Sentence->   तू तू सर्व आहेस

Original English Sentence-->   That day shall come.
Original Marathi Sentence-->   तो दिवस येईलच.
Predicted Marathi Sentence->   ते बंद कर

Original English Sentence-->   I am hers and she is mine.
Original Marathi Sentence-->   मी तिचा आहे व ती माझी.
Predicted Marathi Sentence->   मी माझ्या माझ्या फ्रेंच आहेत

Original English Sentence-->   I'm just pulling your leg.
Original Marathi Sentence-->   मी फक्त तुम्हाला शेंडी लावतेय.
Predicted Marathi Sentence->   मी माझ्या तीन आहेत

Original English Senten

In [28]:
encoding_inference.predict(input_encoder_sentences[1:2])

[array([[ 0.42919114, -0.5411559 ,  0.46467945,  0.54558647, -0.505681  ,
          0.20195134,  0.19051859, -0.5422826 ,  0.30926082, -0.21427141,
         -0.00933504,  0.17363161, -0.2696954 ,  0.6471846 ,  0.40152815,
         -0.36944485,  0.37207875, -0.35169792, -0.5144791 ,  0.09291419,
          0.64537376,  0.6986939 ,  0.26117864,  0.56891596,  0.4500186 ,
         -0.54126036, -0.03436617, -0.02845625,  0.06511852,  0.22114035]],
       dtype=float32),
 array([[ 1.0097774 , -1.7419832 ,  0.8279504 ,  0.7365363 , -2.0253334 ,
          0.39094418,  0.71535456, -0.64492214,  0.5456326 , -0.38805428,
         -0.01888308,  0.31767637, -0.518613  ,  2.6515179 ,  0.6064822 ,
         -0.75984985,  0.84434354, -1.0195284 , -1.4014621 ,  0.10922215,
          1.2187359 ,  1.7618781 ,  0.35226548,  2.5945852 ,  4.048413  ,
         -3.2336683 , -0.06994502, -0.07574804,  0.1498369 ,  0.3013489 ]],
       dtype=float32)]